# Составим список всех возможных значений фильтра

Список возможных значений для поля с диаметров отверстий спишем прям из фильтра с сайта, в нем нет никакой упорядоченности

In [1]:
list_d_otverstiy = [98,100,105,
                    108,110,112,
                   114,114.3,115,118,120,127,130,139,139.7,150,160,170,180,205]

Возможные варианты с отверстиями - это целые числа от 3 до 6. Так как функция range работает в пространстве [аргумент1;аргумент2), и не включает последний аргумент, то делаем диапазон от 3 до 7.

Возможный список вариантов значений для количества отверстий поместим в лист через цикл.

In [2]:
list_otverstiy = [x for x in range(3,7)]

Варианты ширины обода встречаются от 4.5 до 11 с шагом 0.5.

Делаем такой же цикл, как в предыдущем пункте, но внутри range пробежимся по всем значениям, которые в 2 раза больше исходных. 

Не забываем, что range работает не включая идущий последним аргумент, поэтому, чтобы получить максимальное значение в списке 11, делаем второй аргумент в range 11*2+1 = 23

In [3]:
list_shir_oboda = [0.5*x for x in range(9,23)]

Варианты диаметра бывают от 13 до 22. Действуем по аналогии с предыдущими пунктами и создадим список возможных значений фильтра

In [4]:
list_diametr = [x for x in range(13,23)]

# Создадим лист с различными вариантами написания PCD

In [5]:
list_pcd = []
for i in list_otverstiy:
    for j in list_d_otverstiy:
        a = str(i)+'x'+str(j)
        b = str(i)+'х'+str(j)
        c = str(i)+' '+str(j)
        list_pcd.append(a)
        list_pcd.append(b)
        list_pcd.append(c)

# Создадим лист со всеми возможными комбинациями параметров
Этот лист нужен нам для генерации ключевых фраз

Варинаты параметров могут быть:

1) PCD (просто одно значение)

2) PCD + ширина обода

3) PCD + диаметр

4) PCD + ширина обода + диаметр

5) Ширина обода + диаметр

6) Ширина обода (одно значение) 

7) Диаметр (одно значение)

In [6]:
%time
list_pcd4 = []
a = [1,2,3,4,5,6,7]   #просто готовый лист, чтобы промежуточные значения записывать в него
for i in list_shir_oboda:
    for k in list_diametr:
        for j in list_pcd: # пустые значение будем записывать как ; - это разделитель, для колонок таблицы
            a[0] = str(j)+';;'   # PCD (просто одно значение)
            a[1] = str(j)+';'+str(i)+';' # PCD + ширина обода
            a[2] = str(j)+';;'+str(k) # PCD + диаметр
            a[3] = str(j)+';'+str(i)+';'+str(k) # PCD + ширина обода + диаметр
            a[4] = ';'+str(i)+';'+str(k) #  Ширина обода + диаметр
            a[5] = ';'+str(i)+';' # Ширина обода (одно значение) 
            a[6] = ';;'+str(k) # Диаметр (одно значение)
            for x in a:
                list_pcd4.append(x)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.39 µs


Посмотрим сколько возможных комбинаций параметров у нас получилось

In [7]:
len(list_pcd4) 

235200

Всего получили 235 000 ключевых фраз. Немало. Перекинем их в датафрейм через служебный лист csv.
В следующей редакции скрипта сделаю это через словарь.

In [8]:
import pandas as pd
df_1 = pd.DataFrame(data={"col1": list_pcd4})
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235200 entries, 0 to 235199
Data columns (total 1 columns):
col1    235200 non-null object
dtypes: object(1)
memory usage: 1.8+ MB


In [9]:
df_1.to_csv("list_pcd5.csv", sep=',',index=False, encoding='utf-8')

In [10]:
df = pd.read_csv('list_pcd5.csv',sep=',',delimiter=';').reset_index().fillna('')
df.columns = ['pcd','shirina','diametr']
df.head()

,pcd,shirina,diametr
0,3x98,,
1,3x98,4.5,
2,3x98,,13
3,3x98,4.5,13
4,,4.5,13


# Удалим повторяющиеся комбинации параметров

На самом деле, из-за того, чтобы мы делали простые комбинации, а не перестановки параметров фильтра у нас получилось большое количество дубликато, которые надо удалить.

In [11]:
df['concat'] = df['pcd'].astype('str')+df['shirina'].astype('str')+'%'+df['diametr'].astype('str')
df = df.drop_duplicates(subset='concat', keep="first")

In [12]:
df = df.drop(['concat'],axis = 1)

# Переведём числовые колонки в строковый тип, чтобы не получать десятичные дроби в ключевых фразах и параметрах

In [13]:
for i in df.columns:
    df[i] = df[i].astype('str')

In [14]:
df['diametr'] = [x[:-2] for x in df['diametr'].values] #уберём десятичную часть у праметров диаметра

In [15]:
ls = [] #уберём десятичную часть у параметра ширины, если она целая
for x in df['shirina'].values:
    try:
        if x[-1]=='0':
            ls.append(x[:-2])
        else:
            ls.append(x)
    except IndexError:
        ls.append(x)
df['shirina'] = ls
    

# Сгенерируем поисковые запросы

При управлении кампаниями через XLS есть вариант, сразу создавать несколько вариантов ключевых фраз, если по ним не отличается ничего, кроме самой ключевой фразы.

Например такими условиями можно закрывать все возможные варианты написания iphone xs:

(iphone|айфон|ойфон) (xs|+x +s|икс эс|иксэс|хс|+х +с)

Это более короткий вариант записи, чем перечислять 18 возможных комбинаций.
Такой способ работает только при управлении через XLS, и не работает при импорте объявлений через Директ.Коммандер или веб-версию Яндекс.Директа.


In [16]:
#Варианты запросов по-умолчанию (возле диаметра может быть буква R)

df['zapros'] = '(Диски '+df['pcd']+' '+df['shirina']+\
' '+df['diametr']+'|Диски '+df['pcd']+' '+df['shirina']+\
' R'+df['diametr']+')'


In [17]:
# если в поле с диаметром пусто, то не нужно выводить буквы R
df.loc[df['diametr']=='','zapros'] =  'Диски '+df['pcd']+' '+df['shirina']+' '+\
                                   df['diametr']

Сразу посмотрим на самую длинную поисковую фразу, чтобы понять, сколько символов у нас останется в шаблоне объявления.

In [18]:
df['sluzh'] = 'Диски '+df['pcd'].astype('str')+' '+df['shirina'].astype('str')+' R'+\
                                   df['diametr'].astype('str')
qwerty = df['sluzh'].tolist()
qwerty  = [len(x) for x in qwerty]
print(max(qwerty))

22


##### Максимальная длина заголовка - 22 символа

In [19]:
df = df.drop(['sluzh'],axis = 1) #удалили служебную колонку

In [20]:
len('Собственный склад. Более 10 000 колёсных дисков в наличии. На любой кошелёк') #измерили длину текстового описания

75

# Сгенерируем все поля для объявлений

In [ ]:
%%time
df['title1'] = '#Диски# в наличии в Туле' #Заголовок 1
df['title2'] = 'Шинный центр Автошина' #Заголовок 2
df['description'] = 'Собственный склад. Более 10 000 колёсных дисков в наличии. На любой кошелёк.' #Текст объявления
df['bs_title'] = 'Акции||Рассрочка||Контакты||Доставка ' #Заголовки быстрых ссылок, ссылки и описания
df['bs_description'] = 'Актуальные спецпредложения||Процентная ставка 0%. Переплата 0%||Ждём Вас на ул. Рязанской ежедневно||Доставка курьером или самовывоз'
df['bs_links'] = 'http://xn--e1afdpacvp7ap0b.xn--p1ai/actions&utm_source=yandex&utm_medium=cpc&utm_campaign=diski_po_razmery_poisk&utm_term={keyword}||http://xn--e1afdpacvp7ap0b.xn--p1ai/rassrochka?utm_source=yandex&utm_medium=cpc&utm_campaign=diski_po_razmery_poisk&utm_term={keyword}||http://xn--e1afdpacvp7ap0b.xn--p1ai/contacts?utm_source=yandex&utm_medium=cpc&utm_campaign=diski_po_razmery_poisk&utm_term={keyword}||http://xn--e1afdpacvp7ap0b.xn--p1ai/help/delivery?utm_source=yandex&utm_medium=cpc&utm_campaign=diski_po_razmery_poisk&utm_term={keyword}'
df['addictions'] = 'Поиск по авто||Поиск по бренду||Поиск по размеру' #Дополнения

ls = [] # Для генерации параметров получаем из колонки с PCD количество отверстий
for x in df['pcd'].values:
    try:
        ls.append(x[0])
    except IndexError:
        ls.append(x)
df['otverstiya'] = ls


ls = [] # Для генерации параметров получаем из колонки с PCD диаметр отверстий
for x in df['pcd'].values:
    try:
        ls.append(x[2:])
    except IndexError:
        ls.append(x)
df['d_otvertiy'] = ls

# изначально мы хотели сделать весь блок параметров в виде {param1}
# но Яндекс.Директ преобразовывает все спецсимволы в Юникод
# из-за чего сам фильтр переставал работать как набор get-параметров

# пришлось генерировать ссылки с {param1} и {param2}
# при этом параметры PCD мы подставляем в адрес ссылки напрямую, а не через динамические параметры URL
df['link'] = 'http://xn--e1afdpacvp7ap0b.xn--p1ai/discs?tab=param&f[width]={param1}&f[diameter]={param2}&f[et]=&f[pcd_first]='+df['otverstiya']+'&f[pcd_second]='+df['d_otvertiy']+'&utm_source=yandex&utm_medium=cpc&utm_campaign=diski_po_razmery_poisk&utm_term={keyword}'

df['param1'] = df['shirina'] # значения для {param1} и {param2}
df['param2'] = df['diametr']
df['price'] = 3.0 # цена за клик
df['region'] = 'Тула' #регион
#df['id'] = [1+x for x in range(df.shape[0])]
#df['id2'] = [1 if '|' in x else 0 for x in df['zapros'].values]
#df['id3'] = [sum(df['id2'].values[:(x+1)]) for x in range(len(df['id2'].values))]
#df['id4'] = df['id']+df['id3']
#df['group_ids'] = [(x+1)//100 for x in range(df.shape[0])]
df=df.drop(['pcd','shirina','diametr','otverstiya','d_otvertiy'], axis = 1) #удалим служебюные поля

#создадим служебный датафрейм сгруппированный по link
#при помощи этого датафрейма сделаем нумерацию групп объявлений таким образом, 
# чтобы в одной группе было не более 200 ключевых слов
# так как мы генерируем ключевые слова сразу по 2 штуки (с R и без)
# в качестве аргумента для целочисленного деления используем 100

df_sl = df.groupby(by='link',as_index=False).count().reset_index()[['index','link','zapros']]
df_sl.columns = ['index','link','count']
df = df.merge(df_sl,on='link')
for i in df_sl['link'].values:
    df.loc[df['link']==i,'count'] =  [str(df_sl[df_sl['link']==i].values[0][0])+'_'+str((x+1)//100) for x in range(df_sl[df_sl['link']==i].values[0][2])]
df=df.drop(['index'], axis = 1) #убираем служемную колонку 

#### 19 секунд и рекламная кампания готова

In [ ]:
df #посмотрим на данные

## Выгрузим экспортный файл в CSV

In [ ]:
df.to_csv("export_work.csv", sep=',',index=False, encoding='utf-8')

# Что дальше?

#### Отрывем новую книгу в Excel.
#### Переходим на вкладку с данными и нажимаем "Внешние данные" >> "Из текста".

![Alt text](https://monosnap.com/image/04EpRPQvoxAaGQR1yjyKjYx2ZzxDvA.png)

#### Выбираем созданный файл.

![Alt text](https://monosnap.com/image/hGzxYgyJHShowvZtzXEX3vPJtDrLDx.png)

#### Указываем, что в тексте есть разделители и выбираем кодировку UTF-8.

![Alt text](https://monosnap.com/image/0o0axxJgcxP8awCCCO6xai8O3eFofH.png)

#### В разделителях указываем "Запятую" 

![Alt text](https://monosnap.com/image/FQDwy15iXjFuooGa0mi8Y9sgURNtKL.png)

#### Формат даннных "Общий" 

![Alt text](https://monosnap.com/image/KOfExyfTQNlOqZPE98UJetElO3Zuxv.png)

#### После этого MS Excel подумает несколько секунд и создаст нам таблицу.

Остаётся только перенести данные из таблицы в экспортный файл MS Excel и можно загружать рекламную кампанию.

Если на счету недостаточно баллов, то можно попросить их у своего агентского менеджера в Яндексе.  Если же менеджер не достаточно добр - то загружать рекламные кампании придётся частями несколько дней.